In [339]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import numpy as np


In [305]:
df = pd.read_csv(r"C:\Users\Roman Shypka\OneDrive\Робочий стіл\news_data.csv")


In [306]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50030 entries, 0 to 50029
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  50030 non-null  int64 
 1   Text        50027 non-null  object
 2   Label       50030 non-null  bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 830.7+ KB


In [413]:
## clean data from diferent http
df["Text"] = df["Text"].astype(str).str.replace(r'https?://\S+|www\.\S+|bit\.ly\S+',"", regex=True).str.strip()

In [412]:
df[df["Text"].str.contains("https",na=False)]

,Unnamed: 0,Text,Label
12115,12115,7 найцікавіших текстів ТСНua за тижденьСпалах ...,1


In [309]:
# conversion from bool type in int 
le = LabelEncoder()
df["Label"] = le.fit_transform(df["Label"])

In [414]:
df.head(5)

,Unnamed: 0,Text,Label
0,0,В Україні запустять механізм перевірки наявнос...,1
1,1,У Флориді через рекордно низькі температури на...,1
2,2,ТГ-канал FUCKТИ повідомив про бомбардування Ки...,0
3,3,Окупований Маріуполь не здається. Юнак вийшов ...,1
4,4,"Чернігів, Кропивницький і область — повітряна ...",1


In [415]:
df["Label"].value_counts()

Label
1    48006
0     2024
Name: count, dtype: int64

## create balance data

In [312]:
fakes = df[df["Label"] == 0] 

In [313]:
trues = df[df["Label"] == 1 ].sample(n=7000,random_state=42)


In [416]:
df_balanced = pd.concat([trues,fakes]).sample(frac=1,random_state=42).reset_index(drop=True)
df_balanced["Label"].value_counts()

Label
1    7000
0    2024
Name: count, dtype: int64

In [383]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))

## Train

In [384]:
X = (df_balanced["Text"])
y = df_balanced["Label"]

In [385]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [386]:
X_train = vectorizer.fit_transform(X_train_raw)
X_test = vectorizer.transform(X_test_raw)

In [387]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [388]:
predictions = model.predict(X_test)

## Metrics

In [389]:
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

In [390]:
metrics = ["accuracy","precision","recall","f1"]

In [394]:
print(classification_report(y_test, predictions , target_names = ["Fake(0)","True(1)"]))

              precision    recall  f1-score   support

     Fake(0)       0.99      0.63      0.77       437
     True(1)       0.89      1.00      0.94      1368

    accuracy                           0.91      1805
   macro avg       0.94      0.81      0.86      1805
weighted avg       0.92      0.91      0.90      1805



In [441]:
my_text = "Зеленський президент україни телеграм " 

In [442]:
my_vector = vectorizer.transform([my_text])

In [443]:
prediction = model.predict(my_vector)

In [444]:
# model primitive and not smart 
prediction

array([0])

In [445]:
pd.Series(predictions).value_counts()

1    1529
0     276
Name: count, dtype: int64

In [411]:
y_train.value_counts()

Label
1    5632
0    1587
Name: count, dtype: int64

In [419]:
# 1. Retrieve all features (vocabulary) learned by the vectorizer
feature_names = vectorizer.get_feature_names_out()

# 2. Extract weights (coefficients) from the trained logistic regression model
# model.coef_[0] contains the importance values for each feature
coefficients = model.coef_[0]

# 3. Pair each feature name with its corresponding coefficient
word_weights = list(zip(feature_names, coefficients))

# 4. Sort the list by weights (from the most negative to the most positive)
sorted_words = sorted(word_weights, key=lambda x: x[1])

# --- DISPLAYING THE RESULTS ---

print("🛑 TOP-20 words that strongly indicate 'FAKE' (pulling towards 0):")
print("-" * 50)
# Slice the first 20 words with the lowest (negative) coefficients
for word, weight in sorted_words[:20]:
    print(f"{word}: {weight:.4f}")

print("\n ✅ TOP-20 words that strongly indicate 'TRUE' (pulling towards 1):")
print("-" * 50)
# Slice the last 20 words with the highest (positive) coefficients and reverse them
for word, weight in reversed(sorted_words[-20:]):
    print(f"{word}: {weight:.4f}")

🛑 TOP-20 words that strongly indicate 'FAKE' (pulling towards 0):
--------------------------------------------------
телеграм: -7.7016
про це: -6.0191
канали: -4.7637
змі: -4.5522
українські: -4.3787
росії: -4.3354
телеграм канали: -3.6781
каналах: -3.6043
повідомлення: -3.3272
про: -3.0683
це: -3.0306
telegram: -2.9411
російські: -2.9208
поширюється: -2.8752
такі: -2.6878
телеграм каналах: -2.6540
такі повідомлення: -2.5574
пишуть: -2.4398
українські телеграм: -2.3926
повідомляють: -2.2967

 ✅ TOP-20 words that strongly indicate 'TRUE' (pulling towards 1):
--------------------------------------------------
тсн: 4.2081
читайте: 3.1856
далі: 3.1262
читайте далі: 3.0597
окупанти: 2.4218
що: 2.2923
ua: 2.1572
тсн ua: 2.0880
тривога: 2.0733
за: 2.0688
як: 1.8887
повітряна: 1.8479
на: 1.8392
повітряна тривога: 1.8096
youtube тсн: 1.7976
окупантів: 1.7958
ще: 1.7923
також: 1.7851
тсн підтримати: 1.7574
підтримати тсн: 1.7574
